## Packages

In [ ]:
import random
import numpy as np
import os
import torch
import torch.nn as nn
import re
import pandas as pd 
import json
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler
import pickle
from sklearn import metrics
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

In [ ]:
!pip install focal-loss-torch

In [ ]:
from tqdm import tqdm_notebook, trange

def seed_everything(seed = 42): 
  random.seed(seed) 
  os.environ['PYTHONHASHSEED'] = str(seed) 
  np.random.seed(seed)
  torch.manual_seed(seed) 
  torch.cuda.manual_seed(seed) 
  torch.backends.cudnn.deterministic = True
# For reproducible results
seed_everything()

In [ ]:
import matplotlib as mpl
mpl.style.use('seaborn')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
%cd /content/gdrive/My Drive/seq

## Data Preprocessing

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, Y):
        self.data = X
        self.target = Y
        
    def __getitem__(self, index):
        x = self.data[index]
        s = []
        for i in range(types):
          s.append(self.target[index][i])
        
        return x, s
    
    def __len__(self):
        return len(self.data)

### Reading Files

In [ ]:
%ls data/MACS2/

In [ ]:
npzfile = np.load('data/MACS2/merge_c2_neg.npz')

In [ ]:
X, Y = npzfile['arr_0'], npzfile['arr_1']

In [ ]:
X, Y = shuffle(X, Y, random_state=0)

In [ ]:
types = len(Y[0])

In [ ]:
types

In [ ]:
classes = max(max(Y[:,0]), max(Y[:,1])) + 1

### Divide Data

In [ ]:
testX = X[int(len(Y)*0.8):]
testY = Y[int(len(Y)*0.8):]
validX = X[int(len(Y)*0.6):int(len(Y)*0.8)]
validY = Y[int(len(Y)*0.6):int(len(Y)*0.8)]
trainX = X[:int(len(Y)*0.6)]
trainY = Y[:int(len(Y)*0.6)]

In [ ]:
trainX, trainY = shuffle(trainX, trainY, random_state=0)
validX, validY = shuffle(validX, validY, random_state=0)
testX, testY = shuffle(testX, testY, random_state=0)

Data Distrubution for Each Cell

In [ ]:
for i in range(types):
  print(Counter(Y[:,i]))

### Convert to Torch

In [ ]:
train_X = torch.from_numpy(trainX)
train_y = torch.from_numpy(trainY)
valid_X  = torch.from_numpy(validX)
valid_y = torch.from_numpy(validY)
test_X = torch.from_numpy(testX)
test_y = torch.from_numpy(testY)

In [ ]:
train_dataset = MyDataset(train_X, train_y)
valid_dataset = MyDataset(valid_X, valid_y)
test_dataset = MyDataset(test_X, test_y)

## Helper Functions

In [ ]:
def bestmodel(model_name,save_model_time,valid_loss):
    bestloss = 10000
    if valid_loss < bestloss :
        bestloss = valid_loss
        torch.save(model_name, 'model/model{save_model_time}/bestmodel.pkl'.format(save_model_time=save_model_time))
        torch.save(model_name.state_dict(), 'model/model{save_model_time}/net_params_bestmodel.pkl'.format(save_model_time=save_model_time))
    return True  

In [ ]:
def onehot(y):
    y_onehot = np.zeros((len(y), classes), dtype=np.float32)

    all = [i for i in range(classes)]
    for i in range(len(y)):
      y_onehot[i][all.index(y[i])] = 1

    return y_onehot

## Training and Validating

In [ ]:
save_model_time = '0'
mkpath = 'model/model%s'% save_model_time
# os.makedirs(mkpath)

In [ ]:
from focal_loss.focal_loss import FocalLoss

In [ ]:
class TrainHelper():
    '''
    Helper class that makes it a bit easier and cleaner to define the training routine
    
    '''

    def __init__(self,model,train_set,test_set,opts):
      self.model = model  # neural net

      # device agnostic code snippet
      self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      self.model.to(self.device)

      self.epochs = opts['epochs']
      if opts['opt'] == 'Adam':
        self.optimizer = torch.optim.Adam(model.parameters(), opts['lr']) # optimizer method for gradient descent
      else:
        self.optimizer = torch.optim.SGD(model.parameters(), opts['lr'])
      if opts['loss_fxn'] == 'c':
        self.criterion = torch.nn.CrossEntropyLoss()                      # loss function
      elif opts['loss_fxn'] == 'b':
        self.criterion = torch.nn.BCEWithLogitsLoss()                    # loss function used in papers
      elif opts['loss_fxn'] == 'f':
        self.criterion = FocalLoss(alpha=0.25, gamma=2)

      self.train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
      self.valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
    def train(self):
      self.model.train() # put model in training mode
      for epoch in range(self.epochs):
          self.tr_loss = []
          for i, (data,labels) in tqdm_notebook(enumerate(self.train_loader),
                                                  total = len(self.train_loader)):
              label_list = []
              for i in range(len(labels)):
                label_list.append(labels[i].to(self.device))
              data = data.to(self.device)
              self.optimizer.zero_grad()  
              outputs = self.model(data)

              b_list = []
              for i in range(len(label_list)):
                b_list.append(label_list[i])
              if opts['loss_fxn'] != 'c':
                for i in range(len(label_list)):
                  b_list[i] = torch.from_numpy(onehot(labels[i])).to(self.device)

              loss = 0  # define loss
              for i in range(len(outputs)):
                loss += self.criterion(outputs[i], b_list[i])
   
              loss.backward()           
              self.optimizer.step()                  
              self.tr_loss.append(loss.item())       
          if (epoch+1) % 5 == 0 or epoch == 0: # save the model every _ epoch

              torch.save(self.model, 'model/model{save_model_time}/net_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
              torch.save(self.model.state_dict(), 'model/model{save_model_time}/net_params_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
          
          self.test(epoch) # run through the validation set

    def test(self,epoch):
            
      self.model.eval()    # puts model in eval mode
      self.test_loss = []
      self.test_accuracy_L = [[] for _ in range(types)]

      for i, (data, labels) in enumerate(self.valid_loader):
          
          label_list = []
          for i in range(len(labels)):
              label_list.append(labels[i].to(self.device))
          data = data.to(self.device)
          # pass data through network
          # turn off gradient calculation to speed up calcs and reduce memory
          with torch.no_grad():
              outputs = self.model(data)

          # make our predictions and update our loss info
          pred_list = []
          for i in range(len(outputs)):
            _, predicted = torch.max(outputs[i].data, 1)
            pred_list.append(predicted)

          b_list = []
          for i in range(len(label_list)):
            b_list.append(label_list[i])
          if opts['loss_fxn'] != 'c':
            for i in range(len(label_list)):
              b_list[i] = torch.from_numpy(onehot(labels[i])).to(self.device)

          loss = 0  # define loss
          for i in range(len(outputs)):
            loss += self.criterion(outputs[i], b_list[i])

          self.test_loss.append(loss.item())

          for i in range(len(pred_list)):
            self.test_accuracy_L[i].append((pred_list[i] == label_list[i]).sum().item() / pred_list[i].size(0))
      
      test_loss.append(np.mean(self.test_loss))
      train_loss.append(np.mean(self.tr_loss))
      av = [np.mean(self.test_accuracy_L[i]) for i in range(types)]
      bestmodel(self.model,save_model_time,np.mean(self.test_loss)) # find best model
      print('epoch: {}, train loss: {}, test loss: {}, test accuracy: {}'.format( 
            epoch+1, np.mean(self.tr_loss), np.mean(self.test_loss), av))

## Testing

In [ ]:
train_X, train_y = shuffle(train_X, train_y, random_state=0) 
train_X_sub = train_X[:2000]
train_y_sub = train_y[:2000]
sub_dataset = MyDataset(train_X_sub, train_y_sub)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True)

In [ ]:
def test_result(model, datatype):
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True)
    if datatype == 'sub':
      test_loader = torch.utils.data.DataLoader(sub_dataset, batch_size=100, shuffle=True)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    test_accuracy_L = [[] for _ in range(types)]
    f1 = [0 for _ in range(types)]
    recall = [0 for _ in range(types)]
    for i, (data, labels) in enumerate(test_loader):
      label_list = []
      for i in range(len(labels)):
          label_list.append(labels[i].to(device))
      data = data.to(device)
    # pass data through network
    # turn off gradient calculation to speed up calcs and reduce memory
      with torch.no_grad():
          outputs = model(data)
    # make our predictions and update our loss info
      pred_list = []
      for i in range(len(outputs)):
        _, predicted = torch.max(outputs[i].data, 1)
        pred_list.append(predicted)
      for i in range(len(pred_list)):
        test_accuracy_L[i].append((pred_list[i] == label_list[i]).sum().item() / pred_list[i].size(0))
        f1[i] = f1_score(label_list[i].tolist(), pred_list[i].tolist(), average=None)
        recall[i] = recall_score(label_list[i].tolist(), pred_list[i].tolist(), average=None)
    return [np.mean(test_accuracy_L[i]) for i in range(types)], f1, recall

In [ ]:
def pltloss(train_loss, test_loss, epoch):
    epochs = [i for i in range(epoch)]
    fig = plt.figure()
    plt.plot(epochs, train_loss, 'g', label='Training loss')
    plt.plot(epochs, test_loss, 'b', label='Testing loss')
    plt.title('Training and Testing Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

Need better graph

In [ ]:
def pltacc_test(acc_list, epoch):
    epochs = [i for i in range(epoch+1)][::5][1:]
    fig = plt.figure()
    num = acc_list[0]
    acc_list = np.array(acc_list)
    colors = ['g', 'b', 'r', 'c', 'm', 'y', 'k', 'w']
    for i in range(len(num)):
      tmp = 'Testing Accuracy for cell'+str(i+1)
      plt.plot(epochs, acc_list[:,i], colors[i], label=tmp)

    plt.title('Testing Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
 def pltacc_train(acc_list, epoch):
    epochs = [i for i in range(epoch+1)][::5][1:]
    fig = plt.figure()
    num = acc_list[0]
    acc_list = np.array(acc_list)
    colors = ['g', 'b', 'r', 'c', 'm', 'y', 'k', 'w']
    for i in range(len(num)):
      tmp = 'Training Accuracy for cell'+str(i+1)
      plt.plot(epochs, acc_list[:,i], colors[i], linestyle='dashed', label=tmp)

    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

Compare Accuracy Side by Side 

In [ ]:
 def pltacc(tr_acc_1, ts_acc_1, tr_acc_2, ts_acc_2, epoch):
    epochs = [i for i in range(epoch+1)][::5][1:]
    fig = plt.figure()
    plt.plot(epochs, tr_acc_1, 'g', linestyle='dashed', label='Training Accuracy for cell 1')
    plt.plot(epochs, ts_acc_1, 'g', label='Testing Accuracy for cell 1')
    plt.plot(epochs, tr_acc_2, 'b', linestyle='dashed', label='Training Accuracy for cell 2')
    plt.plot(epochs, ts_acc_2, 'b', label='Testing Accuracy for cell 2')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
def confusion(test_data, classifier, num):
    M = np.zeros((classes,classes))
    pred, label = [], []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    for i, (data, labels) in enumerate(test_loader):
      data, labels_i = data.to(device),labels[num].to(device)
      label.extend(labels_i.tolist())
    # pass data through network
    # turn off gradient calculation to speed up calcs and reduce memory
      with torch.no_grad():
        outputs = classifier(data)
    # make our predictions and update our loss info
      _, predicted = torch.max(outputs[num].data, 1)
      pred.extend(predicted.tolist())

    tmp = [i for i in range(classes)]
    M = confusion_matrix(label, pred, labels = tmp)
    print(M.diagonal()/M.sum(axis=1))
    # TN = M[0][0]
    # FN = M[1][0]
    # TP = M[1][1]
    # FP = M[0][1]
    # TPR = TP/(TP+FN)
    # FPR = FP/(FP+TN)
    # print('TPR', TPR)
    # print('FPR', FPR)
    # print('ACC', (TP+TN)/(TP+FP+FN+TN))
    # AUC = metrics.roc_auc_score(label, pred, labels = tmp)
    # print('AUC', AUC)
    # print('\n')

    return M

def visualize_confusion(M):
    fig = plt.figure(figsize = (5, 5))
    ax = fig.add_subplot(1, 1, 1)
    tmp = [i for i in range(classes)]
    cm = ConfusionMatrixDisplay(M, display_labels = tmp);
    cm.plot(values_format = 'd', cmap = 'Blues', ax = ax)

In [ ]:
def getAUC(model, num):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pred, label = [], []
    for i, (data, labels) in enumerate(test_loader):
      label_list = labels[num].to(device)
      label.extend(label_list.tolist())
      data = data.to(device)
    # pass data through network
    # turn off gradient calculation to speed up calcs and reduce memory
      with torch.no_grad():
          outputs = model(data)
    # make our predictions and update our loss info
      _, predicted = torch.max(outputs[num].data, 1)
      pred.extend(predicted.tolist())
    AUC = metrics.roc_auc_score(label, pred, labels=[0,1])
    return AUC

In [ ]:
def getAUPRC(model, num):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pred, label = [], []
    for i, (data, labels) in enumerate(test_loader):
      label_list = labels[num].to(device)
      label.extend(label_list.tolist())
      data = data.to(device)
    # pass data through network
    # turn off gradient calculation to speed up calcs and reduce memory
      with torch.no_grad():
          outputs = model(data)
    # make our predictions and update our loss info
      _, predicted = torch.max(outputs[num].data, 1)
      pred.extend(predicted.tolist())
    auprc = average_precision_score(label, pred)
    return auprc

In [ ]:
def plotAUPRC(model, num):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pred, label = [], []
    for i, (data, labels) in enumerate(test_loader):
      label_list = labels[num].to(device)
      label.extend(label_list.tolist())
      data = data.to(device)
    # pass data through network
    # turn off gradient calculation to speed up calcs and reduce memory
      with torch.no_grad():
          outputs = model(data)
    # make our predictions and update our loss info
      _, predicted = torch.max(outputs[num].data, 1)
      pred.extend(predicted.tolist())
    precision, recall, thresholds = precision_recall_curve(label, pred)
    disp = PrecisionRecallDisplay(precision=precision, recall=recall)
    disp.plot()

## Model

### CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        """
        init convolution and activation layers
        Args:
        x: (Nx4x601)
        class: 

        """
        super(CNN, self).__init__() 
        
        self.conv1 = torch.nn.Conv1d(input_size[0], 32, 3)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv1d(32, 64, 3)
        self.conv3 = torch.nn.Conv1d(64, 64, 3, dilation=1)
        self.pool = torch.nn.MaxPool1d(4)
        self.fc1 = torch.nn.Linear(2304, num_classes)
        self.fc2 = torch.nn.Linear(512, num_classes)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        """
        forward function describes how input tensor is transformed to output tensor
        Args:
            
        """
        # shared layers
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = torch.flatten(x, 1)


        output = []
        # for i in range(2):
        #   tmp = self.conv3(x)
        #   tmp = self.relu(tmp)
        #   tmp = self.pool(tmp)
        #   tmp = torch.flatten(tmp, 1)
        #   tmp = self.fc2(tmp)
        #   tmp = self.sig(tmp)
        #   output.append(tmp)
        for i in range(types):
          tmp = self.fc1(x)
          tmp = self.sig(tmp)
          output.append(tmp)

        return output

In [ ]:
cnn = CNN(train_X.shape[1:], classes)
cnn

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 100,
    'loss_fxn': 'b',
    'opt': 'Adam'
}

In [ ]:
test_loss, train_loss = [], []
CNNTrainer = TrainHelper(model = cnn,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
CNNTrainer.train()

#### Check for Output

In [ ]:
train_acc, tes_acc, tr_f1, tr_recall, ts_f1, ts_recall = [], [], [], [], [], []

In [ ]:
for num in range(opts['epochs']//5):
  cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  cnn.cuda()
  tmp_train, tmp_f1, tmp_r = test_result(cnn, 'sub')
  tr_f1.append(tmp_f1)
  tr_recall.append(tmp_r)
  tmp_test, tmp_f1, tmp_r = test_result(cnn, 'test')
  ts_f1.append(tmp_f1)
  ts_recall.append(tmp_r)
  train_acc.append(tmp_train)
  tes_acc.append(tmp_test)
  print(tmp_train)
  print(tmp_test)

In [ ]:
cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_bestmodel.pkl'))
for i in range(types):
  print(getAUC(cnn, i))

In [ ]:
for i in range(types):
  M = confusion(test_loader, cnn, i)
  visualize_confusion(M)

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltacc_test(tes_acc, opts['epochs'])

In [ ]:
pltacc_train(train_acc, opts['epochs'])

### Basset

In [ ]:
class Basset(nn.Module):
    def __init__(self, input_size, num_class):
        super(Basset, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size[0], out_channels=300, kernel_size=19)
        self.batch1 = nn.BatchNorm1d(num_features=300)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=300, out_channels=200, kernel_size=11)
        self.batch2 = nn.BatchNorm1d(num_features=200)
        self.pool2 = nn.MaxPool1d(kernel_size=4)
        self.conv3 = nn.Conv1d(in_channels=200, out_channels=200, kernel_size=7)

        self.fc1 = nn.Linear(in_features=2000, out_features=1000)
        self.relu4 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.3)

        self.fc2 = nn.Linear(in_features=1000, out_features=1000)
        self.relu5 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.3)

        self.fc3 = nn.Linear(in_features=1000, out_features=num_class)
        self.fc4 = nn.Linear(in_features=164, out_features=2)
        self.sig3 = nn.Sigmoid()

    def forward(self, inputs):
        #output = inputs.unsqueeze(1)
        output = self.conv1(inputs)
        output = self.batch1(output)
        output = self.relu(output)
        output = self.pool1(output)

        output = self.conv2(output)
        output = self.batch2(output)
        output = self.relu(output)
        output = self.pool2(output)


        output = self.conv3(output)
        output = self.batch2(output)
        output = self.relu(output)
        output = self.pool2(output)

        output = torch.flatten(output, 1)
        output = self.fc1(output)
        output = self.relu4(output)
        output = self.dropout1(output)

        output = self.fc2(output)
        output = self.relu5(output)
        output = self.dropout2(output)

        x = []
        for i in range(types):
          tmp = self.fc3(output)
          tmp = self.sig3(tmp)
          x.append(tmp)

        # output = self.fc3(output)
        # output = self.sig3(output)
        # output = self.fc4(output)

        return x

In [ ]:
basset = Basset(train_X.shape[1:], classes)
basset

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 25,
    'batch_size': 100,
    'loss_fxn': 'c',
    'opt': 'SGD'
}

In [ ]:
test_loss, train_loss = [], []
BassetTrainer = TrainHelper(model = basset,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
BassetTrainer.train()

#### Check for Output

In [ ]:
train_acc, tes_acc, tr_f1, tr_recall, ts_f1, ts_recall = [], [], [], [], [], []

In [ ]:
for num in range(opts['epochs']//5):
  basset.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  basset.cuda()
  tmp_train, tmp_f1, tmp_r = test_result(basset, 'sub')
  tr_f1.append(tmp_f1)
  tr_recall.append(tmp_r)
  tmp_test, tmp_f1, tmp_r = test_result(basset, 'test')
  ts_f1.append(tmp_f1)
  ts_recall.append(tmp_r)
  train_acc.append(tmp_train)
  tes_acc.append(tmp_test)
  print(tmp_train)
  print(tmp_test)

In [ ]:
ts_f1

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltacc_train(train_acc, opts['epochs'])

In [ ]:
pltacc_test(tes_acc, opts['epochs'])

In [ ]:
basset.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_1.pkl'))

In [ ]:
for i in range(types):
  M = confusion(test_loader, basset, i)
  visualize_confusion(M)

### LeNet

In [ ]:
class LeNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LeNet, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_size[0], out_channels=6, kernel_size=5) 
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool2 = nn.MaxPool1d(kernel_size=4)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=120, kernel_size=5)

        self.fc1 = nn.Linear(in_features=8280, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)

        x = self.relu(self.conv2(x))
        x = self.pool2(x)

        x = self.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        # print(x.size())
        # assert 0
        output = []
        for i in range(types):
          tmp = self.fc1(x)
          tmp = self.fc2(tmp)
          output.append(tmp)

        return output

In [ ]:
lenet = LeNet(train_X.shape[1:], classes)
lenet

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 100,
    'loss_fxn': 'c',
    'opt': 'Adam'
}
test_loss, train_loss = [], []
LeNetTrainer = TrainHelper(model = lenet,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
LeNetTrainer.train()